In [1]:
import pandas as pd
import datetime as dt
import os
import glob

In [2]:
def get_filenames(path):
    '''
    Given a file path with wildcard for extension, capture all filenames.
    '''
    
    file_list = {}
    file_list = glob.glob(path)
    
    return file_list

In [4]:
def agency_short(df):
    agency_dict = {'Department of Veterans Affairs': 'VA',
                'Department of Labor': 'Labor',
                'General Services Administration' : 'GSA',
                'Department of Defense': 'DoD',
                'US Agency for International Development' : 'USAID',
                'Department of State' : 'State',
                'Nuclear Regulatory Commission' : 'NRC',
                'Department of Homeland Security' : 'DHS',
                'Department of Education' : 'ED',
                'Department of Justice' : 'DOJ',
                'National Science Foundation' : 'NSF',
                'Office of Personnel Management' : 'OPM',
                'National Aeronautics and Space Administration' : 'NASA',
                'Department of Health and Human Services' : 'HHS',
                'Environmental Protection Agency' : 'EPA',
                'Department of Commerce' : 'DOC',
                'Department of Transportation' : 'DOT',
                'Social Security Administration' : 'SSA',
                'Department of Agriculture' : 'USDA',
                'Department of Housing and Urban Development' : 'HUD',
                'Department of Energy' : 'DOE',
                'Department of the Treasury' : 'Treasury',
                'Small Business Administration' : 'SBA',
                'Department of the Interior' : 'DOI'}
    
    df['Agency Short Name'] = df['Agency'].map(agency_dict)
    
    return df

In [5]:
def PleaseChoose_clean(df):
    PC_dict = {
        '[Please Choose]' : 'No Response'}
    df = df.replace(PC_dict)
    
    return df

## Systems

In [6]:
def get_systems_data(path):
    file_list = get_filenames(path)
    sheet = "Part IIA Systems Inventory"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        try:
            temp_df['f) Anticipated Upgrade Date or End of Life'] = pd.to_datetime(temp_df['f) Anticipated Upgrade Date or End of Life'])
        except KeyError:
            temp_df['f) Anticipated Upgrade Date or End of Life'] = []
        temp_df['Anticipated Upgrade Date FY or End of Life FY'] = pd.DatetimeIndex(temp_df['f) Anticipated Upgrade Date or End of Life']).year

        li.append(temp_df)
        
    temp_df = pd.concat(li, axis=0, ignore_index=True, sort=False)
    
    temp_df['d) Is this a shared system?'] = temp_df['d) Is this a shared system?'].fillna(temp_df['c) Is this a shared system?'])
    temp_df['c) Name of Product'] = temp_df['c) Name of Product'].fillna(temp_df['d) Name of Product'])
    temp_df = temp_df.drop(['c) Is this a shared system?','d) Name of Product',
                           'f) Anticipated Upgrade Date or End of Life'], axis=1)
    
    temp_df['j) Investment in solution is >$500,000'] = temp_df['j) Investment in solution is >$500,000'].replace('Yes','TRUE')

    temp_df = temp_df.rename(index=str, columns={
        "a) Function (Choose from drop-down)": "Function",
        "b) Name of System or Tool": "System Name",
        "d) Is this a shared system?": "Shared System (y/n)",
        "c) Name of Product": "Software Name",
        "e) Version of Product": "Version",
        "g) Age of Current System": "System Age",
        "h) As applicable, UII for CPIC investment": "UII or CPIC",
        "i) As applicable, please add the FY 2018 PIID for any contracts supporting this production instance (services and systems)": "2018 PIID",
        "j) Investment in solution is >$500,000": "Investment Greater than $500k",
        "k) Please rate customer satisfaction with this system (5=Very Satisfied, 1=Very Dissatisfied)": "Customer Satisfaction Rating",
        "l) Would you like to consider an alternative system?": "Consider Alternative"
        })
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
        
    return temp_df

In [7]:
systems_df = get_systems_data("data/2019/*.xls")

In [8]:
try:
    systems_df.to_csv("data/2019//Final Data/systems.csv", encoding = 'utf-8')
except FileNotFoundError:
    os.makedirs("data/2019//Final Data")
    systems_df.to_csv("data/2019//Final Data/systems.csv", encoding = 'utf-8')

## Functions

In [12]:
def get_functions_data(path):
    file_list = get_filenames(path)
    sheet = "Part IA Functions Activities"

    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        temp_df = temp_df.rename(index=str, columns={
            "a) If agency Grants Management staff perform the activity, please provide the count of these staff performing this activity as measured in annual FTE*": "Grants Mgmt FTE in Function",
            "b) If agency Grants Program staff perform the activity, please provide the count of these staff performing the activity as measured in annual FTE*": "Grants Program FTE in Function",
            "c) If an agency Centralized Processing Office performs this activity, please provide the count of staff performing this activity as measured in annual FTE*": "Centralized SSP FTE in Function",
            "d) If a contractor performs the activity, please provide the number of Contract Staff providing this activity as measured in annualized contract staff*": "Contractor FTE in Function",
            "e) If a Centralized Processing Office performs the activity, please provide the names of the organization(s)/office(s) providing processing services.": "Centralized SSP Office/Agency",
            "f) Is this activity standardized? Yes/No": "Standardized (y/n)",
            "g) If this activity is not standardized, please explain.": "Reason not Standardized",
            "h) Would the agency like to consider an outside provider for this activity?": "Consider External Provider"
        })
        
        
        temp_df = agency_short(temp_df)
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [13]:
functions_df = get_functions_data("data/2019/*.xls")

In [15]:
functions_df.to_csv("data/2019//Final Data/functions.csv", encoding = 'utf-8')

## Additional Capabilities

In [16]:
def get_capabilities_data(path):
    file_list = get_filenames(path)
    sheet = "Part IB Additional Capabilities"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True, sort=False)
    temp_df = temp_df.drop(['Agency POC Email','Agency Point of Contact (POC) '], axis=1)
    
    temp_df = temp_df.rename(index=str, columns={
        "ADDITIONAL Business Capability Statement": "Additional Business Capability Statement",
        "Authoritative Reference for THIS Business Capability Statement (required for must have)": "Authoritative Reference for THIS Business Capability Statement"
        })
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [17]:
capabilities_df = get_capabilities_data("data/2019/*.xls")

In [18]:
capabilities_df.to_csv("data/2019//Final Data/capabilities.csv", encoding = 'utf-8')

## Questions

In [19]:
def get_comments_data(path):
    file_list = get_filenames(path)
    sheet_list = ["Part IIB - Readiness for Change",
                 "Part III Question #2",
                 "4) Standard Data Definitions",
                 "Part III Question #1",
                 "3) Organizational Environment"]
    li = []
    for file_name in file_list:
        for sheet in sheet_list:
            temp_df = pd.read_excel(file_name, sheet_name = sheet)
            temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
            temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
            temp_df = temp_df.rename(index=str, columns={temp_df.columns[0]: "Question",
                                                        temp_df.columns[1]: "Response",
                                                        temp_df.columns[2]: "Recommendations/Concerns?"})
            
            li.append(temp_df)
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [20]:
question_df =  get_comments_data("data/2019/*.xls")

In [21]:
question_df = question_df.drop_duplicates()

In [22]:
question_df.to_csv("data/2019//Final Data/questions.csv", encoding = 'utf-8')

## Comments/Suggestions

In [23]:
def get_comments_data(path):
    file_list = get_filenames(path)
    sheet = "7) Comments"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        temp_df['Type'] = 'Comment'
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    temp_df = temp_df.rename(index=str, columns={
        "7) General Comments": "Value"})
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [24]:
def get_suggestions_data(path):
    file_list = get_filenames(path)
    sheet = "2) Suggestions"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        temp_df['Type'] = 'Suggestion'
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    temp_df = temp_df.rename(index=str, columns={
        "2) Do you have any suggestions for improving existing shared grants systems that you use?": "Value"})
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [25]:
comments_df = get_comments_data("data/2019/*.xls")

In [26]:
suggestions_df = get_suggestions_data("data/2019/*.xls")

In [27]:
feedback_df = comments_df.append(suggestions_df)      

In [28]:
feedback_df.to_csv("data/2019//Final Data/feedback.csv", encoding = 'utf-8')

## Pain Points

In [29]:
def get_pain_points_data(path):
    file_list = get_filenames(path)
    sheet = "5) Pain Points"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    temp_df = temp_df.rename(index=str, columns={
        "Pain Points": "Pain Point"})
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [30]:
pain_df = get_pain_points_data("data/2019/*.xls")

In [31]:
pain_df.to_csv("data/2019//Final Data/pain points.csv", encoding = 'utf-8')

try:
    capabilities_df.to_csv("data/2019//Final Data/capabilities.csv", encoding = 'utf-8')
except FileNotFoundError:
    os.makedirs("data/2019//Final Data")
    capabilities_df.to_csv("data/2019//Final Data/capabilities.csv", encoding = 'utf-8')

## Policy Impediments

In [32]:
def get_policy_imped_data(path):
    file_list = get_filenames(path)
    sheet = "6) Policy Impediments"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        
        li.append(temp_df)
    
    temp_df = pd.concat(li, axis=0, ignore_index=True, sort=False)
    temp_df = temp_df.rename(index=str, columns={
        "Name of Policy": "Policy Name",
        "Choose from drop downs below": "Impediment Type"})
    temp_df = temp_df.drop(['6) If applicable, please provide details of any statute, regulation or policy impeding the further adoption, standardization, or consolidation including name, type, and any additional relevant information.'], axis=1)
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [33]:
policy_df = get_policy_imped_data("data/2019/*.xls")

In [34]:
policy_df.to_csv("data/2019//Final Data/policy impediments.csv", encoding = 'utf-8')

## Metrics

In [35]:
def std_cap_ratio(df):
    #denominator is total number of Activities for Grants Management (15) 
    y_denominator = 15
    try:
        y_numerator = df['Must Have/Nice to Have'].value_counts().loc['Must Have']
    except KeyError:
        y_numerator = 0
    
    try: 
        standard_capabilities_ratio = float(y_numerator) / y_denominator
    except ZeroDivisionError:
        standard_capabilities_ratio = None
    
    return standard_capabilities_ratio

In [36]:
def num_req_cap(df):
    try:
        req_cap = df['Must Have/Nice to Have'].value_counts().loc['Must Have']
    except KeyError:
        req_cap = None
    
    return req_cap

In [37]:
def standardization_score(row):
    if row['f) Is this activity standardized? Yes/No'] == 'Yes':
        return 100
    if row['f) Is this activity standardized? Yes/No'] == 'No':
        return 0
    if row['f) Is this activity standardized? Yes/No'] == 'In Part':
        return 50
    else:
        None

In [38]:
def stand_score_calc(file_name):
    
    sheet = "Part IA Functions Activities"
    temp_df = pd.read_excel(file_name, sheet_name = sheet)
    temp_df['Standardization Score'] = temp_df.apply (lambda row: standardization_score(row), axis=1)
    
    from pandas.core.groupby.groupby import DataError
    try:
        weighted_std_score = temp_df.groupby(by = ['Activity', 'Function'])['Standardization Score'].mean().mean()
    except DataError:
        weighted_std_score = None 

    return weighted_std_score

In [39]:
def get_metrics_data(path):
    file_list = get_filenames(path)
    sheet = "Part IB Additional Capabilities"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        agency = os.path.basename(file_name).split(' - ')[1]
        service_area = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        standard_capabilities_ratio = std_cap_ratio(temp_df)
        req_cap = num_req_cap(temp_df)
        stand_score = stand_score_calc(file_name)
        try:
            stand_factor = (standard_capabilities_ratio)/2 + (stand_score)/2
        except TypeError:
            stand_factor = None
        
        d = {'Agency': [agency], 
             'Service Area': [service_area],
             'Standardized Factor': [stand_factor],
             'Number of Required Capabilities': [req_cap],
             'Standard Capabilities Ratio': [standard_capabilities_ratio],
             'Inter-Agency Aligment': [stand_score]}
        temp_df = pd.DataFrame(data = d)
        temp_df = pd.melt(temp_df, id_vars = ['Agency', 'Service Area'], 
        value_vars=['Inter-Agency Aligment', 'Number of Required Capabilities','Standard Capabilities Ratio' ,'Standardized Factor'], 
        var_name='Variable', value_name='Value')
        
        li.append(temp_df)
        
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [40]:
metrics_df = get_metrics_data("data/2019/*.xls")

In [41]:
metrics_df.to_csv("data/2019//Final Data/metrics.csv", encoding = 'utf-8')

## Standardization

In [42]:
def get_standardization_data(path):
    file_list = get_filenames(path)
    sheet = "Part IA Functions Activities"
    li = []
    for file_name in file_list:
        temp_df = pd.read_excel(file_name, sheet_name = sheet)
        temp_df['Agency'] = os.path.basename(file_name).split(' - ')[1]
        temp_df['Service Area'] = (os.path.basename(file_name.split(' - ')[0])).replace(' Readiness Assessment', '')
        temp_df['Standardization Score'] = temp_df.apply (lambda row: standardization_score(row), axis=1)
        temp_df = temp_df[['Agency','Service Area','Function', 'Activity','Standardization Score']]
        
        li.append(temp_df)
        
    temp_df = pd.concat(li, axis=0, ignore_index=True)
    
    temp_df = agency_short(temp_df)
    temp_df = PleaseChoose_clean(temp_df)
    
    return temp_df

In [43]:
standardization_df = get_standardization_data("data/2019/*.xls")

In [44]:
standardization_df.to_csv("data/2019//Final Data/standardization.csv", encoding = 'utf-8')